<div align="center">
    <span style="font-size:28px; font-weight:bold; color:rgba(49, 248, 109, 0.84)">TƯƠNG TÁC DỮ LIỆU TRỰC QUAN</span><br>
</div>

---

## **Tiền xử lý dữ liệu**

#### **2.1. Khai báo thư viện**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import warnings
import plotly.express as px
import plotly.io as pio
import country_converter as coco
import us
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.renderers.default = 'notebook'
warnings.filterwarnings('ignore')

#### **2.2. Đọc tập dữ liệu**

In [2]:
df = pd.read_excel('E-Commerce data for regression.xlsx')
df.head(5)

,Order ID,Order Date,Ship Date,Aging,Ship Mode,Product Category,Product,Sales,Quantity,Discount,...,Shipping Cost,Order Priority,Customer ID,Customer Name,Segment,City,State,Country,Region,Months
0,AU-2015-1,2015-11-09,2015-11-17,8,First Class,Auto & Accessories,Car Media Players,140,2,0.05,...,4.600,Medium,LS-001,Lane Daniels,Consumer,Brisbane,Queensland,Australia,Oceania,Nov
1,AU-2015-2,2015-06-30,2015-07-02,2,First Class,Auto & Accessories,Car Speakers,211,3,0.03,...,11.201,Medium,IZ-002,Alvarado Kriz,Home Office,Berlin,Berlin,Germany,Central,Jun
2,AU-2015-3,2015-12-05,2015-12-13,8,First Class,Auto & Accessories,Car Body Covers,117,5,0.01,...,3.115,Critical,EN-003,Moon Weien,Consumer,Porirua,Wellington,New Zealand,Oceania,Dec
3,AU-2015-4,2015-05-09,2015-05-16,7,First Class,Auto & Accessories,Car & Bike Care,118,2,0.05,...,2.620,High,AN-004,Sanchez Bergman,Corporate,Kabul,Kabul,Afghanistan,Central Asia,May
4,AU-2015-5,2015-07-09,2015-07-18,9,First Class,Auto & Accessories,Tyre,250,1,0.04,...,16.000,Critical,ON-005,Rowe Jackson,Corporate,Townsville,Queensland,Australia,Oceania,Jul


#### **2.3. Kiểm tra số dòng, số cột, kiểu dữ liệu từng cột**

In [3]:
print("\nThông tin cơ bản về dữ liệu:")
print(df.info())


Thông tin cơ bản về dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          51290 non-null  object        
 1   Order Date        51290 non-null  datetime64[ns]
 2   Ship Date         51290 non-null  datetime64[ns]
 3   Aging             51290 non-null  int64         
 4   Ship Mode         51290 non-null  object        
 5   Product Category  51290 non-null  object        
 6   Product           51290 non-null  object        
 7   Sales             51290 non-null  int64         
 8   Quantity          51290 non-null  int64         
 9   Discount          51290 non-null  float64       
 10  Profit            51290 non-null  float64       
 11  Shipping Cost     51290 non-null  float64       
 12  Order Priority    51290 non-null  object        
 13  Customer ID       51290 non-null  object      

#### **2.4. Chuyển các cột có kiểu 'object' thành 'string**

In [4]:
for column in df.select_dtypes(include=['object']).columns:
    df[column] = df[column].astype('string')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order ID          51290 non-null  string        
 1   Order Date        51290 non-null  datetime64[ns]
 2   Ship Date         51290 non-null  datetime64[ns]
 3   Aging             51290 non-null  int64         
 4   Ship Mode         51290 non-null  string        
 5   Product Category  51290 non-null  string        
 6   Product           51290 non-null  string        
 7   Sales             51290 non-null  int64         
 8   Quantity          51290 non-null  int64         
 9   Discount          51290 non-null  float64       
 10  Profit            51290 non-null  float64       
 11  Shipping Cost     51290 non-null  float64       
 12  Order Priority    51290 non-null  string        
 13  Customer ID       51290 non-null  string        
 14  Customer Name     5129

#### **2.5. Kiểm tra giá trị thiếu**

In [5]:
null_quantity = df.isnull().sum()
null_percentages = df.isnull().mean() * 100

duplicated_null_stats = pd.DataFrame({
    'Số lượng giá trị null': null_quantity,
    'Tỷ lệ giá trị null (%)': null_percentages
}).sort_values(by='Tỷ lệ giá trị null (%)', ascending=False)

print("Thông tin về missing value:")
print ("-"*80)
display(duplicated_null_stats)

Thông tin về missing value:
--------------------------------------------------------------------------------


,Số lượng giá trị null,Tỷ lệ giá trị null (%)
Order ID,0,0.0
Shipping Cost,0,0.0
Region,0,0.0
Country,0,0.0
State,0,0.0
City,0,0.0
Segment,0,0.0
Customer Name,0,0.0
Customer ID,0,0.0
Order Priority,0,0.0


#### **2.6. Kiểm tra giá trị trùng lặp**

In [6]:
print("\nSố dòng dữ liệu bị trùng lặp:")
print(df.duplicated().sum())


Số dòng dữ liệu bị trùng lặp:
0


#### **2.7. Kiểm tra giá trị ngoại lai**

In [7]:
numeric_cols = df[['Sales', 'Quantity', 'Discount', 'Profit', 'Shipping Cost', 'Aging']]

outlier_summary = pd.DataFrame(columns=['Q1', 'Q3', 'IQR', 'Số lượng ngoại lai'])

for col in numeric_cols.columns:
    Q1 = numeric_cols[col].quantile(0.25)
    Q3 = numeric_cols[col].quantile(0.75)
    IQR = Q3 - Q1

    outliers = ((numeric_cols[col] < (Q1 - 1.5 * IQR)) | (numeric_cols[col] > (Q3 + 1.5 * IQR)))
    outlier_count = outliers.sum()
    
    outlier_summary.loc[col] = [Q1, Q3, IQR, outlier_count]

print("Tóm tắt ngoại lai theo tứ phân vị:")
print("-" * 80)
print(outlier_summary.sort_values(by='Số lượng ngoại lai', ascending=False))

Tóm tắt ngoại lai theo tứ phân vị:
--------------------------------------------------------------------------------
                    Q1       Q3      IQR  Số lượng ngoại lai
Sales          109.000  218.000  109.000                 0.0
Quantity         2.000    4.000    2.000                 0.0
Discount         0.020    0.040    0.020                 0.0
Profit          25.730  120.560   94.830                 0.0
Shipping Cost    2.573   12.056    9.483                 0.0
Aging            3.000    8.000    5.000                 0.0


#### **2.8. Thêm thuộc tính mới**

In [8]:
df['Total'] = (df['Sales'] * df['Quantity']*(1 - df['Discount'])) + df['Shipping Cost']

df['Year'] = df['Order Date'].dt.year
df['Month_Num'] = df['Order Date'].dt.month

df['Profit_Margin'] = df['Profit'] / df['Sales']
df['Profit_Margin'] = df['Profit_Margin'].replace([float('inf'), -float('inf')], pd.NA)

cc = coco.CountryConverter()
df['Region Grouped'] = cc.convert(names=df['Country'], to='continent')

#### **2.9. Chuẩn hóa tên cột**

In [9]:
df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

#### **2.10. Hiển thị dữ liệu sau tiền xử lý**

In [10]:
df.head(5)

,order_id,order_date,ship_date,aging,ship_mode,product_category,product,sales,quantity,discount,...,city,state,country,region,months,total,year,month_num,profit_margin,region_grouped
0,AU-2015-1,2015-11-09,2015-11-17,8,First Class,Auto & Accessories,Car Media Players,140,2,0.05,...,Brisbane,Queensland,Australia,Oceania,Nov,270.600,2015,11,0.328571,Oceania
1,AU-2015-2,2015-06-30,2015-07-02,2,First Class,Auto & Accessories,Car Speakers,211,3,0.03,...,Berlin,Berlin,Germany,Central,Jun,625.211,2015,6,0.530853,Europe
2,AU-2015-3,2015-12-05,2015-12-13,8,First Class,Auto & Accessories,Car Body Covers,117,5,0.01,...,Porirua,Wellington,New Zealand,Oceania,Dec,582.265,2015,12,0.266239,Oceania
3,AU-2015-4,2015-05-09,2015-05-16,7,First Class,Auto & Accessories,Car & Bike Care,118,2,0.05,...,Kabul,Kabul,Afghanistan,Central Asia,May,226.820,2015,5,0.222034,Asia
4,AU-2015-5,2015-07-09,2015-07-18,9,First Class,Auto & Accessories,Tyre,250,1,0.04,...,Townsville,Queensland,Australia,Oceania,Jul,256.000,2015,7,0.640000,Oceania


<br><br>

---